In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from kernels import polynomial_kernel
from perceptrons import OneVsAllKernelPerceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [128]:
df = pd.read_csv('zipcombo.dat', sep=' ', header=None).drop(columns=[257])
df.rename(columns={0: 'label'}, inplace=True)
X = df[list(range(1, 257))].values
y = df['label'].values.astype(np.int)

In [24]:
def subsample(df, classes, sample_size=100):
    # sampling
    df_small = pd.DataFrame()
    for clazz in classes:
        df_clazz = df[df['y'] == clazz]
        df_sample = df_clazz.sample(sample_size)
        df_small = df_small.append(df_sample)

    #shuffle
    df_small = df_small.sample(frac=1.)

    X_small = df_small.drop(columns='y').values
    y_small = df_small['y'].values
    
    return X_small, y_small

In [79]:
df = pd.DataFrame(X)
df['y'] = y
X, y = subsample(df, list(range(10)), sample_size=100)

In [4]:
def one_hot(y_train, y_test):
    classes = 10
    values_train = y_train.reshape(-1)
    values_test = y_test.reshape(-1)
    enc_y_train = np.eye(classes)[values_train]
    enc_y_test = np.eye(classes)[values_test]
    return enc_y_train, enc_y_test

Write own k nearest neighbours algorithm

In [5]:
def knn(X_train, y_train, X_test, k):
    distances = np.zeros(len(X_train))
    ks_idx = np.zeros(k)
    #generate list of distances from test point
    for i in range(0, len(X_train)):
        distances[i] = np.linalg.norm(X_train[i] - X_test)
    
    #pick top k points from list
    ks = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
    return ks

Calculate priors

In [6]:
def priors(y_train, y_test):
    e_y_train, e_y_test = one_hot(y_train, y_test)
    ph1 = (1 + e_y_train.sum(axis=0))/(1*2+len(e_y_train))
    ph0 = 1 - ph1
    return ph1, ph0

Calculate posteriors

In [85]:
def posterior_fastest(X_train, y_train, y_test, k):
    peH1 = np.zeros((10, k+1))
    peH0 = np.zeros((10, k+1))
    kmodel = KNeighborsClassifier(k+1)
    kmodel.fit(X_train, y_train)
    neighbs_list = []
    for i in range(len(X_train)):
        neighbs_list.append(kmodel.kneighbors(X_train[i].reshape(1, -1), k+1)[1][0])
    neighbs_list = np.delete(neighbs_list, 0, 1)
    e_y_train, e_y_test = one_hot(y_train, y_test)
    for lab in range(e_y_train.shape[1]):
        c1 = np.zeros(k+1)
        c0 = np.zeros(k+1)
        for i in range((len(X_train))):
            neighbs = neighbs_list[i]
            deltas = 0
            for a in neighbs:
                if e_y_train[a][lab] == 1:
                    deltas += 1
            if e_y_train[i][lab] == 1.0:
                c1[deltas] += 1
            else:
                c0[deltas] += 1

        for k in range(0, k+1):
            peH1[lab][k] = (1 + c1[k]) / ((k+1) + c1.sum())
            peH0[lab][k] = (1 + c0[k]) / ((k+1) + c0.sum())
    return peH1, peH0

Training run fastest

In [130]:
def training_run(X_train, X_test, y_train, y_test, k, run_type='basic'):
    #calc priors
    ph1, ph0 = priors(y_train, y_test)
    #calc posteriors
    peH1, peH0 = posterior_fastest(X_train, y_train, y_test, k)
    #run training iteration
    label_pred_test = np.zeros((len(y_test), 10))
    label_pred_train = np.zeros((len(y_train), 10))
    e_y_train, e_y_test = one_hot(y_train, y_test)
    kmodel_test = KNeighborsClassifier(k+1)
    kmodel_test.fit(X_train, y_train)
    train_list = []
    test_list = []
    
    if run_type == 'basic':
        for i in tqdm(range(0,len(y_train))):
            train_list.append(kmodel_test.kneighbors(X_train[i].reshape(1, -1), k+1)[1][0])
        np.delete(train_list, 0, 1)
        for i in range(0, len(y_train)):
            kss = train_list[i]
            for l in range(e_y_train.shape[1]):
                c = 0
                for ks in range(k):
                    if e_y_train[kss[ks]][l] == 1:
                        c += 1
                y1 = ph1[l]*peH1[l][c]
                y0 = ph0[l]*peH0[l][c]
                if y1 > y0:
                    label_pred_train[i][l] = 1
                else:
                    label_pred_train[i][l] = 0
    else:
    
        for i in range(0,len(y_test)):
            test_list.append(kmodel_test.kneighbors(X_test[i].reshape(1, -1), k+1)[1][0])
        np.delete(test_list, 0, 1)
        for i in range(0, len(y_test)):
            kss = test_list[i]
            for l in range(e_y_train.shape[1]):
                c = 0
                for ks in range(k):
                    if e_y_train[kss[ks]][l] == 1:
                        c += 1
                y1 = ph1[l]*peH1[l][c]
                y0 = ph0[l]*peH0[l][c]
                if y1 > y0:
                    label_pred_test[i][l] = 1
                else:
                    label_pred_test[i][l] = 0
    
    return label_pred_train, label_pred_test

In [134]:
training_run(X_train, X_test, y_train, y_test, 1, run_type='cv')

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]]))

Do basic runs

In [147]:
# perform basic runs
iterations = 5
list_ks = [1, 2, 3, 4]
err_train = {k: [] for k in list_ks}
err_test = {k: [] for k in list_ks}

for iteration in tqdm(list(range(iterations))):
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)
    
    for k in list_ks:
        y_pred_train, y_pred_test = training_run(X_train, X_test, y_train, y_test, k)
        score_train = np.sum(np.argmax(y_pred_train, axis=1) == y_train)
        score_test = np.sum(np.argmax(y_pred_test, axis=1) == y_test)
        err_rate_train = ((len(y_train) - score_train)/len(y_train))*100
        err_rate_test = ((len(y_test) - score_test)/len(y_test))*100
        err_train[k].append(err_rate_train)
        err_test[k].append(err_rate_test)
    
err_train_mean = {k: np.mean(errs) for k, errs in err_train.items()}
err_test_mean = {k: np.mean(errs) for k, errs in err_test.items()}
err_train_std = {k: np.std(errs) for k, errs in err_train.items()}
err_test_std = {k: np.std(errs) for k, errs in err_test.items()}









  0%|                                                                                            | 0/5 [00:00<?, ?it/s]








  0%|                                                                                         | 0/7438 [00:00<?, ?it/s]








  0%|▏                                                                              | 21/7438 [00:00<00:36, 205.22it/s]








  1%|▍                                                                              | 47/7438 [00:00<00:34, 211.47it/s]








  1%|▋                                                                              | 69/7438 [00:00<00:34, 212.66it/s]








  1%|▉                                                                              | 89/7438 [00:00<00:35, 207.46it/s]








  1%|█                                                                             | 107/7438 [00:00<00:37, 196.63it/s]








  2%|█▎                                                                            | 124/7438 [0

 18%|█████████████▌                                                               | 1308/7438 [00:06<00:34, 176.98it/s]








 18%|█████████████▊                                                               | 1330/7438 [00:06<00:32, 186.70it/s]








 18%|█████████████▉                                                               | 1352/7438 [00:06<00:31, 194.69it/s]








 18%|██████████████▏                                                              | 1374/7438 [00:07<00:30, 200.70it/s]








 19%|██████████████▍                                                              | 1395/7438 [00:07<00:31, 192.97it/s]








 19%|██████████████▋                                                              | 1416/7438 [00:07<00:30, 196.25it/s]








 19%|██████████████▉                                                              | 1439/7438 [00:07<00:29, 203.20it/s]








 20%|███████████████                                                              | 1461/7438 [00:07<00:

 36%|███████████████████████████▊                                                 | 2689/7438 [00:13<00:27, 173.92it/s]








 36%|████████████████████████████                                                 | 2709/7438 [00:13<00:26, 178.94it/s]








 37%|████████████████████████████▎                                                | 2729/7438 [00:13<00:25, 184.38it/s]








 37%|████████████████████████████▍                                                | 2748/7438 [00:14<00:29, 160.75it/s]








 37%|████████████████████████████▋                                                | 2766/7438 [00:14<00:28, 164.83it/s]








 37%|████████████████████████████▊                                                | 2783/7438 [00:14<00:34, 135.83it/s]








 38%|████████████████████████████▉                                                | 2801/7438 [00:14<00:31, 146.37it/s]








 38%|█████████████████████████████▏                                               | 2819/7438 [00:14<00:

 53%|████████████████████████████████████████▉                                    | 3955/7438 [00:20<00:18, 186.18it/s]








 53%|█████████████████████████████████████████▏                                   | 3975/7438 [00:20<00:18, 189.19it/s]








 54%|█████████████████████████████████████████▎                                   | 3995/7438 [00:20<00:19, 177.63it/s]








 54%|█████████████████████████████████████████▌                                   | 4014/7438 [00:21<00:19, 179.26it/s]








 54%|█████████████████████████████████████████▊                                   | 4033/7438 [00:21<00:18, 181.45it/s]








 54%|█████████████████████████████████████████▉                                   | 4052/7438 [00:21<00:19, 177.89it/s]








 55%|██████████████████████████████████████████▏                                  | 4070/7438 [00:21<00:19, 174.01it/s]








 55%|██████████████████████████████████████████▎                                  | 4089/7438 [00:21<00:

 71%|██████████████████████████████████████████████████████▍                      | 5253/7438 [00:27<00:13, 158.80it/s]








 71%|██████████████████████████████████████████████████████▌                      | 5270/7438 [00:27<00:13, 161.49it/s]








 71%|██████████████████████████████████████████████████████▋                      | 5287/7438 [00:27<00:13, 163.43it/s]








 71%|██████████████████████████████████████████████████████▉                      | 5304/7438 [00:28<00:12, 164.30it/s]








 72%|███████████████████████████████████████████████████████                      | 5321/7438 [00:28<00:12, 165.91it/s]








 72%|███████████████████████████████████████████████████████▎                     | 5338/7438 [00:28<00:12, 165.16it/s]








 72%|███████████████████████████████████████████████████████▍                     | 5357/7438 [00:28<00:12, 171.14it/s]








 72%|███████████████████████████████████████████████████████▋                     | 5375/7438 [00:28<00:

...

 57%|███████████████████████████████████████████▊                                 | 4238/7438 [00:20<00:15, 204.41it/s]








 57%|████████████████████████████████████████████                                 | 4259/7438 [00:20<00:15, 205.69it/s]








 58%|████████████████████████████████████████████▎                                | 4280/7438 [00:20<00:15, 198.00it/s]








 58%|████████████████████████████████████████████▌                                | 4301/7438 [00:21<00:15, 201.15it/s]








 58%|████████████████████████████████████████████▊                                | 4324/7438 [00:21<00:15, 201.18it/s]








 58%|████████████████████████████████████████████▉                                | 4345/7438 [00:21<00:15, 202.35it/s]








 59%|█████████████████████████████████████████████▏                               | 4366/7438 [00:21<00:15, 204.23it/s]








 59%|█████████████████████████████████████████████▍                               | 4387/7438 [00:21<00:

 75%|█████████████████████████████████████████████████████████▋                   | 5570/7438 [00:27<00:10, 183.66it/s]








 75%|█████████████████████████████████████████████████████████▊                   | 5589/7438 [00:27<00:10, 180.90it/s]








 75%|██████████████████████████████████████████████████████████                   | 5611/7438 [00:27<00:09, 185.26it/s]








 76%|██████████████████████████████████████████████████████████▎                  | 5631/7438 [00:27<00:09, 187.45it/s]








 76%|██████████████████████████████████████████████████████████▌                  | 5655/7438 [00:27<00:09, 194.39it/s]








 76%|██████████████████████████████████████████████████████████▋                  | 5675/7438 [00:28<00:09, 192.08it/s]








 77%|██████████████████████████████████████████████████████████▉                  | 5695/7438 [00:28<00:09, 190.66it/s]








 77%|███████████████████████████████████████████████████████████▏                 | 5715/7438 [00:28<00:

 91%|██████████████████████████████████████████████████████████████████████▏      | 6779/7438 [00:34<00:03, 169.33it/s]








 91%|██████████████████████████████████████████████████████████████████████▎      | 6797/7438 [00:34<00:03, 172.04it/s]








 92%|██████████████████████████████████████████████████████████████████████▌      | 6815/7438 [00:34<00:03, 172.49it/s]








 92%|██████████████████████████████████████████████████████████████████████▋      | 6833/7438 [00:34<00:03, 172.80it/s]








 92%|██████████████████████████████████████████████████████████████████████▉      | 6854/7438 [00:34<00:03, 180.74it/s]








 92%|███████████████████████████████████████████████████████████████████████▏     | 6875/7438 [00:35<00:02, 187.74it/s]








 93%|███████████████████████████████████████████████████████████████████████▎     | 6894/7438 [00:35<00:02, 188.02it/s]








 93%|███████████████████████████████████████████████████████████████████████▌     | 6914/7438 [00:35<00:

  8%|█████▉                                                                        | 565/7438 [00:02<00:37, 184.91it/s]








  8%|██████                                                                        | 584/7438 [00:03<00:36, 186.01it/s]








  8%|██████▎                                                                       | 606/7438 [00:03<00:35, 194.01it/s]








  8%|██████▌                                                                       | 628/7438 [00:03<00:34, 199.12it/s]








  9%|██████▊                                                                       | 649/7438 [00:03<00:36, 184.97it/s]








  9%|███████                                                                       | 668/7438 [00:03<00:36, 184.97it/s]








  9%|███████▏                                                                      | 689/7438 [00:03<00:35, 191.00it/s]








 10%|███████▍                                                                      | 713/7438 [00:03<00:

KeyboardInterrupt: 

In [135]:
# display in dataframe
df_err = pd.DataFrame([err_train_mean, err_test_mean,
                       err_train_std, err_test_std], 
                       index=['train_error_mean', 'test_error_mean', 'train_error_std', 'test_error_std'], 
                       columns=list_ks).T
df_err

,train_error_mean,test_error_mean,train_error_std,test_error_std
1,0.000,90.9,0.000000,0.734847
2,3.350,90.9,0.906228,0.734847
3,5.300,90.9,0.930726,0.734847
4,5.775,90.9,0.755811,0.734847


Perform cross validation setup

In [136]:
def make_fold_indices(n, k=5):
    ixs = np.array(range(n))
    np.random.shuffle(ixs)
    folds = np.array_split(ixs, k)
    fold_ixs = np.zeros(n)
    for i in range(k):
        fold_ixs[folds[i]] = i
    return fold_ixs

In [142]:
# generate k folds and perform cross-validation on them, returning error per fold.
def cross_validation_error_knn(X, y, neighbs, k=5):
    fold_ixs = make_fold_indices(len(X), k=k)

    cv_errs = []
    for fold_ix in tqdm(np.unique(fold_ixs)):
        X_val = X[fold_ixs == fold_ix]
        y_val = y[fold_ixs == fold_ix]
        X_train = X[fold_ixs != fold_ix]
        y_train = y[fold_ixs != fold_ix]
        
        #record validation fold error
        y_pred_train, y_pred_test = training_run(X_train, X_val, y_train, y_val, neighbs, run_type='cv')
        cv_errs.append(np.sum(np.argmax(y_pred_test, axis=1) == y_val))
        
    return np.mean(cv_errs)

Perform cross validation runs

In [143]:
# perform cross-validation runs

iterations = 5
list_ks = [1, 2, 3, 4, 5]
errs_cv = {}
k_stars = []
errs_test = []
for iteration in tqdm(list(range(iterations))):
    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2)
    
    # perform cross validations
    for ks in list_ks:
        errs_cv[ks] = cross_validation_error_knn(X_train, y_train, ks)
        
    # get best parameter
    k_star = max(errs_cv, key=errs_cv.get)
    k_stars.append(k_star)
    
    # get final error
    y_pred_train, y_pred_test = training_run(X_train, X_test, y_train, y_test, d_star, run_type='cv')
    accuracy = np.sum(np.argmax(y_pred_test, axis=1) == y_test)
    err_rate = ((len(y_test) - accuracy)/len(y_test))*100
    errs_test.append(err_rate)
    
# compute results   
err_test_mean = np.mean(errs_test)
d_star_mean = np.mean(k_stars)
err_test_std = np.std(errs_test)
d_star_std = np.std(k_stars)







  0%|                                                                                            | 0/5 [00:00<?, ?it/s]






  0%|                                                                                            | 0/5 [00:00<?, ?it/s]






 20%|████████████████▊                                                                   | 1/5 [00:26<01:47, 26.99s/it]






 40%|█████████████████████████████████▌                                                  | 2/5 [00:53<01:20, 26.88s/it]






 60%|██████████████████████████████████████████████████▍                                 | 3/5 [01:20<00:53, 26.79s/it]






 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:47<00:26, 26.88s/it]






100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:14<00:00, 26.89s/it]






  0%|                                                                                            | 0/5 [00:00<?,

KeyboardInterrupt: 

In [144]:
# display in dataframe
df_err = pd.DataFrame([[err_test_mean, err_test_std],
                       [d_star_mean, d_star_std]], 
                       columns=['mean', 'std'], index=['error_test', 'k_star']).T
print("Answer to 2:")
df_err

Answer to 2:


,error_test,k_star
mean,"{1: 90.9, 2: 90.9, 3: 90.9, 4: 90.9}",1.6
std,"{1: 0.7348469228349535, 2: 0.7348469228349535,...",0.8
